<a href="https://colab.research.google.com/github/alif-munim/language-models/blob/main/lora/svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Singular Value Decomposition
To understand LoRA, we first need to understand the rank of matrices and singular value decomposition.

In [1]:
import torch
import numpy as np
_ = torch.manual_seed(0)

In [3]:
"""
The (d, k) matrix represents one hidden layer for a model with input dim d
and hidden dim k.
"""

d, k = 10, 10
W_rank = 2

"""
We multiply a (d, 2) matrix by a (2, k) matrix to create a (d, k) matrix
This new matrix has many linearly dependent columns (redundancy) and has rank 2
"""
W = torch.randn(d, W_rank) @ torch.randn(W_rank, k)
print(W)

tensor([[ 2.8501, -4.1679, -1.2931, -1.7376, -2.5698, -3.2220, -1.4271, -1.2982,
          0.2702,  1.2163],
        [ 3.2737, -4.7411, -1.4644, -1.9621, -2.9216, -3.6760, -1.6166, -1.4949,
          0.2975,  1.3819],
        [-0.0141, -3.3560, -1.5177, -2.4550, -2.1852, -1.7979, -1.6433,  0.2801,
          0.9375,  1.1010],
        [-0.8365,  0.4910,  0.0490, -0.0243,  0.2776,  0.5523,  0.0609,  0.4404,
          0.1243, -0.1169],
        [-3.9740, -0.6857, -1.1295, -2.3176, -0.6460,  1.0025, -1.1858,  2.3367,
          1.4298,  0.4341],
        [ 0.7376, -0.9989, -0.2987, -0.3915, -0.6132, -0.7910, -0.3304, -0.3424,
          0.0478,  0.2886],
        [-2.2472,  1.8582,  0.3750,  0.3281,  1.0966,  1.7733,  0.4272,  1.1393,
          0.1840, -0.4908],
        [ 0.7821, -0.5984, -0.1087, -0.0790, -0.3502, -0.5912, -0.1251, -0.4004,
         -0.0775,  0.1550],
        [-0.0482, -0.4016, -0.1912, -0.3150, -0.2638, -0.1991, -0.2066,  0.0602,
          0.1267,  0.1342],
        [ 0.6151, -

In [4]:
# We can evaluate the rank of the new matrix with numpy
W_rank = np.linalg.matrix_rank(W)
print(f'Rank of W: {W_rank}')

Rank of W: 2


In [7]:
"""
Now, we can decompose the rank-deficient matrix W using singular-value decomposition (SVD).
SVD produces 3 matrices U, S, and V which produce W when multiplied.
The dimension of USV, however, is much smaller than W.
"""

U, S, V = torch.svd(W)

# For rank r factorization, we keep only the first r singular values and corresponding columns of U and V
S_r = torch.diag(S[:W_rank])
U_r = U[:, :W_rank]
V_r = V[:, :W_rank].t()

# The operator @ performs matrix multiplication
B = U_r @ S_r
A = V_r
print(f'Shape of A: {A.shape}')
print(f'Shape of B: {B.shape}')

Shape of A: torch.Size([2, 10])
Shape of B: torch.Size([10, 2])


In [9]:
"""
We can compare the decomposed matrices and the full matrix W by simulating a neural network
with random inputs.
"""

x = torch.randn(d)
bias = torch.randn(d)

y_W = W @ x + bias
y_BA = (B @ A) @ x + bias

print(f'Original output using W:\n{y_W}\n')
print(f'Recomputed output using low-rank matrices A and B:\n{y_W}\n')

Original output using W:
tensor([ 5.2066,  5.4057,  2.7002,  0.3157, -0.6567,  1.3392, -1.8407,  0.2161,
         0.3822,  0.2234])

Recomputed output using low-rank matrices A and B:
tensor([ 5.2066,  5.4057,  2.7002,  0.3157, -0.6567,  1.3392, -1.8407,  0.2161,
         0.3822,  0.2234])



In [10]:
"""
Finally, we compare the total number of parameters.
"""

print("Total params of W: ", W.nelement())
print("Total params of B and A: ", B.nelement() + A.nelement())

Total params of W:  100
Total params of B and A:  40
